In [ ]:
import duckdb
import pandas as pd
# No need to import duckdb_engine
#  jupysql will auto-detect the driver needed based on the connection string!

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql


parquet_file_path = '/Users/safmomen/Downloads/deere_dbx_audit/*.parquet'



In [ ]:
%config SqlMagic.autopandas = False
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%sql duckdb:///:memory:


In [ ]:
%%sql
CREATE view usage AS SELECT * FROM read_parquet('/Users/safmomen/Downloads/deere_dbx_audit/*.parquet');


In [ ]:
%%sql
select count(*) num_records
from
usage

In [ ]:
%%sql
describe
select *
from
usage;

In [ ]:
%%sql
select *
from
usage
limit 10

In [ ]:
%%sql
summarize
select *
from
usage
limit 100000

In [ ]:
%%sql
df <<
select 
    "requestParams.jobTaskType",
    count(*) cnt
from
usage
group by 1


In [ ]:
display(df)
df.bar()

In [ ]:
%%sql
df <<
select 
    "requestParams.jobTerminalState",
    count(*) cnt
from
usage
group by 1


In [ ]:
display(df)
df.bar()

In [ ]:
%%sql
df <<
select 
    "actionName",
    count(*) cnt
from
usage
group by 1
order by 2


In [ ]:
display(df)
df.bar()

In [ ]:
%%sh 
jupyter nbconvert --to html loader.ipynb --output deere_contents.html

In [ ]:
raise KeyboardInterrupt


In [ ]:
some missing stuff:
    
if using databrickssql:
    https://docs.databricks.com/api-explorer/workspace/queryhistory/list

https://docs.databricks.com/administration-guide/account-settings/usage-analysis.html
https://docs.databricks.com/administration-guide/account-settings/billable-usage-download-api.html
https://docs.databricks.com/administration-guide/account-settings/audit-logs.html

In [ ]:

duckdb.sql(
    f"""
    select 
        "requestParams.runId"
    from '{parquet_file_path}' 
    limit 100000"""
)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%reload_ext dotenv

%dotenv ../../env/.env

import warnings
import utils
from pandas import Timedelta
import time
from utils import logger, sql_to_df, run_sql, session, conn
import pandas as pd
warnings.filterwarnings('ignore')

try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime

In [ ]:
run_sql("use role dev")

In [ ]:
# Define the source Parquet file path on your local machine
# parquet_file_path = '/path/to/your/parquet/file.parquet'

parquet_file_paths = [
    '/Users/safmomen/Downloads/deere_dbx_audit/part-00000-tid-7465844095593919475-c056804b-1121-4690-86d2-013c692cfda9-5-1-c000.snappy.parquet',
    '/Users/safmomen/Downloads/deere_dbx_audit/part-00001-tid-7465844095593919475-c056804b-1121-4690-86d2-013c692cfda9-6-1-c000.snappy.parquet',
    '/Users/safmomen/Downloads/deere_dbx_audit/part-00002-tid-7465844095593919475-c056804b-1121-4690-86d2-013c692cfda9-7-1-c000.snappy.parquet'
]

# Define the target Snowflake stage and the file name in the stage
stage_name = 'raw_customer.deere.raw'

for p in parquet_file_paths:
    # Execute the PUT command to upload the Parquet file to the stage
    put_command = f"PUT file://{p} @{stage_name}/files/"
    conn.cursor().execute(put_command)